In [116]:
import warnings
warnings.filterwarnings('ignore')

In [117]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [118]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [119]:
# Load the cleaned data into a data frame
file_path = Path('./cleaned_data/data_by_country.csv')
df = pd.read_csv(file_path)
#df = pd.read_csv(file_path, index_col=0)
df.head(20)            

,Country,Literacy_Rate,Buddhists,Christians,Folk_Religions,Hindus,Jews,Muslims,Other_Religions,Unaffiliated,...,gpd_2010,gpd_2015,gdp_per_capita_2000,gdp_per_capita_2005,gdp_per_capita_2010,gdp_per_capita_2015,mortality_rate_2000,mortality_rate_2005,mortality_rate_2010,mortality_rate_2015
0,Albania,96.8,1.0,18.0,1.0,1.0,1.0,80.3,1.0,1.4,...,1.192692e+10,1.138685e+10,1126.683340,2673.786584,4094.348386,3952.802538,106.10,105.70,103.50,92.24
1,Algeria,72.6,1.0,1.0,1.0,1.0,1.0,97.9,1.0,1.8,...,1.612070e+11,1.659790e+11,1765.027146,3113.094883,4480.786318,4177.889542,141.90,119.30,104.40,96.49
2,Angola,70.4,1.0,90.5,4.2,1.0,1.0,1.0,1.0,5.1,...,8.169956e+10,8.721929e+10,556.836182,1902.422150,3497.974488,3127.890598,383.60,322.70,269.70,239.80
3,Antigua and Barbuda,99.0,1.0,93.0,3.6,1.0,1.0,1.0,1.0,1.7,...,1.148700e+09,1.336693e+09,10872.292950,12557.547850,13048.960580,14285.329780,156.70,156.10,134.60,121.90
4,Argentina,97.9,1.0,85.2,1.0,1.0,1.0,1.0,1.0,12.2,...,4.236270e+11,5.947490e+11,7708.099115,5109.852245,10385.964430,13789.060420,138.50,124.90,118.70,112.70
5,Armenia,99.6,1.0,98.5,1.0,1.0,1.0,1.0,1.0,1.3,...,9.260285e+09,1.055334e+10,622.740923,1643.756889,3218.378299,3607.289299,143.40,139.70,137.90,121.30
6,Australia,99.0,2.7,67.3,1.0,1.4,1.0,2.4,1.0,24.2,...,1.147590e+12,1.350530e+12,21697.708480,34080.999900,52087.972290,56707.022080,76.93,68.13,63.06,62.17
7,Austria,98.0,1.0,80.4,1.0,1.0,1.0,5.4,1.0,13.5,...,3.922750e+11,3.819710e+11,24625.600720,38417.457790,46903.761590,44195.817590,93.72,83.70,74.43,64.34
8,Azerbaijan,99.8,1.0,3.0,1.0,1.0,1.0,96.9,1.0,1.0,...,5.290929e+10,5.307437e+10,655.119945,1578.402390,5843.533768,5500.310382,161.00,160.00,131.80,119.50
9,Bahrain,94.6,2.5,14.5,1.0,9.8,1.0,70.3,1.0,1.9,...,2.571327e+10,3.105064e+10,13636.416750,17959.396830,20722.070490,22634.085650,100.80,83.48,65.97,58.52


In [120]:
high = 20
med = 10
# Turn mortality_rate_2015 into a %
df['mortality_rate'] = df['mortality_rate_2015'].div(1000).mul(100)

# Syntax to add column based on another column range found at
#https://stackoverflow.com/questions/59642338/creating-new-column-based-on-condition-on-other-column-in-pandas-dataframe
df['mortality_state']=np.select([df['mortality_rate']<med,df['mortality_rate'].between(med,high)],[1,2],3)
df.head(10)


,Country,Literacy_Rate,Buddhists,Christians,Folk_Religions,Hindus,Jews,Muslims,Other_Religions,Unaffiliated,...,gdp_per_capita_2000,gdp_per_capita_2005,gdp_per_capita_2010,gdp_per_capita_2015,mortality_rate_2000,mortality_rate_2005,mortality_rate_2010,mortality_rate_2015,mortality_rate,mortality_state
0,Albania,96.8,1.0,18.0,1.0,1.0,1.0,80.3,1.0,1.4,...,1126.683340,2673.786584,4094.348386,3952.802538,106.10,105.70,103.50,92.24,9.224,1
1,Algeria,72.6,1.0,1.0,1.0,1.0,1.0,97.9,1.0,1.8,...,1765.027146,3113.094883,4480.786318,4177.889542,141.90,119.30,104.40,96.49,9.649,1
2,Angola,70.4,1.0,90.5,4.2,1.0,1.0,1.0,1.0,5.1,...,556.836182,1902.422150,3497.974488,3127.890598,383.60,322.70,269.70,239.80,23.980,3
3,Antigua and Barbuda,99.0,1.0,93.0,3.6,1.0,1.0,1.0,1.0,1.7,...,10872.292950,12557.547850,13048.960580,14285.329780,156.70,156.10,134.60,121.90,12.190,2
4,Argentina,97.9,1.0,85.2,1.0,1.0,1.0,1.0,1.0,12.2,...,7708.099115,5109.852245,10385.964430,13789.060420,138.50,124.90,118.70,112.70,11.270,2
5,Armenia,99.6,1.0,98.5,1.0,1.0,1.0,1.0,1.0,1.3,...,622.740923,1643.756889,3218.378299,3607.289299,143.40,139.70,137.90,121.30,12.130,2
6,Australia,99.0,2.7,67.3,1.0,1.4,1.0,2.4,1.0,24.2,...,21697.708480,34080.999900,52087.972290,56707.022080,76.93,68.13,63.06,62.17,6.217,1
7,Austria,98.0,1.0,80.4,1.0,1.0,1.0,5.4,1.0,13.5,...,24625.600720,38417.457790,46903.761590,44195.817590,93.72,83.70,74.43,64.34,6.434,1
8,Azerbaijan,99.8,1.0,3.0,1.0,1.0,1.0,96.9,1.0,1.0,...,655.119945,1578.402390,5843.533768,5500.310382,161.00,160.00,131.80,119.50,11.950,2
9,Bahrain,94.6,2.5,14.5,1.0,9.8,1.0,70.3,1.0,1.9,...,13636.416750,17959.396830,20722.070490,22634.085650,100.80,83.48,65.97,58.52,5.852,1


In [121]:
# Set y
y = df[['mortality_state']]

# Drop Country column - non numeric AND unneeded
# Drop Mortality Columns as we will use 1 target = mortality_rate_2015
df = df.drop(['Country','mortality_rate_2000',
              'mortality_rate_2005','mortality_rate_2010',
              'mortality_rate_2015','mortality_rate','mortality_state'], axis=1)
# Set X
X=df

In [122]:
X.describe()

,Literacy_Rate,Buddhists,Christians,Folk_Religions,Hindus,Jews,Muslims,Other_Religions,Unaffiliated,alcohol_rate_2000,...,pg_2010,pg_2015,gdp_2000,gpd_2005,gpd_2010,gpd_2015,gdp_per_capita_2000,gdp_per_capita_2005,gdp_per_capita_2010,gdp_per_capita_2015
count,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,139.000000,...,139.000000,139.000000,1.390000e+02,1.390000e+02,1.390000e+02,1.390000e+02,139.000000,139.000000,139.000000,139.000000
mean,84.597122,4.627338,56.810072,2.492086,3.816547,1.536691,25.540288,1.126619,8.372662,6.075086,...,1.607997,1.508579,1.349748e+11,1.941989e+11,2.939314e+11,3.278879e+11,6721.934067,10399.044353,13445.236083,13556.432606
std,18.451735,15.201296,37.196149,4.840336,11.054644,6.327489,36.371413,0.820443,11.906068,4.343943,...,1.691831,1.262347,4.899748e+11,5.767136e+11,8.434781e+11,1.095522e+12,10176.117043,15825.146048,19210.410630,18671.255326
min,28.700000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.038000,...,-2.096943,-0.940754,2.048496e+08,2.617976e+08,3.668400e+08,4.370062e+08,124.460791,151.681566,234.235539,305.511144
25%,74.350000,1.000000,13.200000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.440000,...,0.486291,0.580451,3.026418e+09,6.196934e+09,9.993191e+09,1.138751e+10,618.236654,995.758723,1554.010559,2087.257010
50%,93.100000,1.000000,71.200000,1.000000,1.000000,1.000000,4.600000,1.000000,3.000000,5.480000,...,1.354308,1.301979,1.056658e+10,1.700346e+10,3.219727e+10,4.578013e+10,2001.540049,3193.204358,5076.339872,5967.052204
75%,98.950000,1.000000,89.150000,1.500000,1.000000,1.000000,46.950000,1.000000,10.550000,9.015000,...,2.581132,2.490801,7.384932e+10,1.173875e+11,1.927675e+11,1.975390e+11,7368.332777,11159.221660,14365.283865,15385.623900
max,100.000000,96.900000,99.000000,35.600000,80.700000,75.600000,99.000000,9.700000,59.600000,17.450000,...,11.483371,5.790591,4.968360e+12,4.831470e+12,6.087160e+12,1.106160e+13,48659.598880,80988.137620,110885.991400,105462.012600


In [123]:
# Check the balance of our target values
y['mortality_state'].value_counts()

2    58
1    42
3    39
Name: mortality_state, dtype: int64

In [124]:
# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [125]:
print(X_train.shape)

(104, 25)


In [126]:
print(y_train.shape)

(104, 1)


## Easy Ensemble Classifier

In [127]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_e_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easy_e_model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [128]:
# Calculated the balanced accuracy score
y_pred = easy_e_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6833333333333332

In [129]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 8,  0,  0],
       [ 2, 12,  1],
       [ 0,  9,  3]], dtype=int64)

In [130]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.80      1.00      0.93      0.89      0.96      0.93         8
          2       0.57      0.80      0.55      0.67      0.66      0.45        15
          3       0.75      0.25      0.96      0.38      0.49      0.22        12

avg / total       0.68      0.66      0.78      0.62      0.67      0.48        35



## Balanced Random Forest Classifier

In [131]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Create a balanced random forest classifier.
brf_model = BalancedRandomForestClassifier(n_estimators = 100, random_state=1)

# Fitting the model
brf_model = brf_model.fit(X_train, y_train)
brf_model

BalancedRandomForestClassifier(random_state=1)

In [132]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = brf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7472222222222222

In [133]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 7,  1,  0],
       [ 1, 13,  1],
       [ 1,  5,  6]], dtype=int64)

In [134]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          1       0.78      0.88      0.93      0.82      0.90      0.81         8
          2       0.68      0.87      0.70      0.76      0.78      0.62        15
          3       0.86      0.50      0.96      0.63      0.69      0.46        12

avg / total       0.76      0.74      0.84      0.73      0.78      0.61        35

